# Office

In [23]:
import pandas as pd

## Database connection

In [24]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [25]:
sede = pd.read_sql_table("sede", OLTP_connection)
city = pd.read_sql_table("ciudad", OLTP_connection)
region = pd.read_sql_table("departamento", OLTP_connection)

## Transformation

In [26]:
office_dimension = sede.merge(city, on="ciudad_id", how="left")

In [27]:
office_dimension.rename(
	columns={
		"sede_id": "original_office_id",
		"nombre_x": "office_name",
		"nombre_y": "office_city"
	}, inplace=True
)

In [ ]:
office_dimension = office_dimension[[
			"original_office_id",
			"office_name",
			"office_city",
			"departamento_id"
		]]

In [29]:
office_dimension = office_dimension.merge(region, on="departamento_id", how="left")

office_dimension.drop(columns=["departamento_id"], inplace=True)
office_dimension.rename(columns={ "nombre": "office_region" }, inplace=True)

In [80]:
office_dimension.reset_index(inplace=True)
office_dimension.rename(columns={ "index": "office_id" }, inplace=True)
office_dimension.set_index("office_id", inplace=True)

## Load

In [82]:
office_dimension.to_sql(
	"OFFICE_DIMENSION", OLAP_connection, if_exists="replace", index=True
)

52